In [1]:

import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
from xskillscore import pearson_r
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cmaps
import cartopy.feature as cf
from xskillscore import pearson_r_p_value

import cartopy.crs as ccrs
import cartopy.feature as cf
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as colors
import math

In [5]:
# Define the GCMs
experiments = ["EC-Earth3","GFDL-CM4","INM-CM4-8",
               "IPSL-CM6A-LR","MIROC6","MPI-ESM1-2-HR",
               "MRI-ESM2-0", "NorESM2-LM"]

gauges_file = '/Users/yjshen/Documents/00fish-mwork/Nicole_thesis/Flood-extreme/gcmdata/info_gis_selected_47datapoints.csv'
grdc_csv = pd.read_csv(gauges_file, index_col=None)
print(grdc_csv)

grdc_csv['ix'] = grdc_csv['longitude'].apply(lambda x: math.floor((x + 180) / 0.25))
grdc_csv['iy'] = grdc_csv['latitude'].apply(lambda x: math.floor((90 - x) / 0.25))

# note that upperleft corner, (0,0) is (-180 179.75), (90, 89.75) >>> python index starts from 0
# I am not sure how the nc file stored the data, I mean if nc file contains data like this: (0,0) is (-180 179.75), (90, 89.75) >>> Correct
# if nc : (0,0) is (-180 179.75), (-70, -59.75) <<<< means the first elements is downside. You need to add this code to fliup the data.

grdc_csv['iy'] = 600 - grdc_csv['iy'] + 1  

# Yes, nc stored file like this:
# South 60-59.75, [data1, data2 data3, ... ,data4, data5]   <<<<<<<< -180W, 180E
# South 59.75-59.5, [data1, data2 data3, ... ,data4, data5] <<<<<<<< -180W, 180E
# .....
# .....
# .....
# North 89.75-90, [data1, data2 data3, ... ,data4, data5]   <<<<<<<< -180W, 180E


for i in range(8):
    TAG = experiments[i]

    ds1 = xr.open_dataset(f'/Users/yjshen/Documents/00fish-mwork/Nicole_thesis/Flood-extreme/gcmdata/nc_file/{TAG}_1850-2015.nc')

    data1 = ds1['discharge'].values

    x_l, y_l = grdc_csv['ix'], grdc_csv['iy']
    # print(x_l)
    # # print(len(x_l),len(y_l))
    # # print(data1)
    
    natout = data1[:,y_l, x_l]
    # print(natout.shape)
    # print(natout)
    
    NAT_FILE = '/Users/yjshen/Documents/00fish-mwork/Nicole_thesis/Flood-extreme/gcmdata/allrivers_' + TAG + '.csv'
    pd.DataFrame(natout).to_csv(NAT_FILE)

     id  longitude  latitude
0     1     -90.37    38.869
1     2     -91.08    33.969
2     3     -95.19    35.458
3     4     -91.62    31.146
4     5     -95.90    41.103
5     6    -121.30    61.801
6     7    -122.20    55.999
7     8    -117.34    46.591
8     9    -139.96    64.310
9    10    -132.47    60.037
10   11     -64.89    -3.154
11   12     -54.54    -2.370
12   13     -58.34   -10.366
13   14      -0.48    16.878
14   15      18.25     0.100
15   16      22.06     4.138
16   17      17.39    -3.310
17   18      23.11   -14.208
18   19      26.96    -5.898
19   20      20.61   -26.987
20   21      26.84   -23.851
21   22     147.50   -30.162
22   23     151.14   -29.574
23   24     149.02   -34.082
24   25     147.69   -36.082
25   26     149.22   -23.028
26   27     105.36    21.190
27   28     105.47    15.231
28   29     103.04    18.132
29   30     100.42    24.365
30   31     106.77    29.539
31   32     112.41    29.814
32   33     112.73    28.834
33   34     11